In [ ]:
import pandas as pd


In [ ]:
questions_df = pd.read_csv("/content/CHATBOT CAPSTONE DATASET.csv", encoding='unicode_escape')


In [ ]:
answers_df = pd.read_csv("/content/CHATBOT CAPSTONE DATASET.csv", encoding='unicode_escape')


In [ ]:
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
import re

def preprocess(text):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    text = re.sub(r'[^\w\s]', '', text)  # Remove non-alphanumeric characters
    tokens = nltk.word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]
    return ' '.join(stemmed_tokens)





In [ ]:
def preprocess_with_stopwords(text):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    text = re.sub(r'[^\w\s]', '', text)  # Remove non-alphanumeric characters
    tokens = nltk.word_tokenize(text.lower())
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]
    return ' '.join(stemmed_tokens)



In [ ]:
questions_df = pd.read_csv("/content/CHATBOT CAPSTONE DATASET.csv", encoding='unicode_escape')  # Specify the encoding
questions_list = questions_df['Questions'].tolist()  # Assuming 'Question' is the column name containing questions

In [ ]:
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords


nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab') # Download the missing 'punkt_tab' resource

def preprocess(text):
    # Check if the input is a string, if not convert it
    if not isinstance(text, str):
        text = str(text)
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    text = re.sub(r'[^\w\s]', '', text)  # Remove non-alphanumeric characters
    tokens = nltk.word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]
    return ' '.join(stemmed_tokens)


vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize)
X = vectorizer.fit_transform([preprocess(q) for q in questions_list]) # This line caused the error

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
def get_response(user_query):
    processed_text = preprocess_with_stopwords(user_query)
    vectorized_text = vectorizer.transform([processed_text])
    similarities = cosine_similarity(vectorized_text, X)
    max_similarity = np.max(similarities)

    if max_similarity > 0.6:
        # Find high similarity questions
        high_similarity_indices = [i for i, s in enumerate(similarities[0]) if s > 0.6]
        high_similarity_questions = [questions_list[i] for i in high_similarity_indices]

        # Retrieve corresponding answers
        target_answers = [answers_list[i] for i in high_similarity_indices]

        # Further refine based on corrected text
        Z = vectorizer.transform([preprocess_with_stopwords(q) for q in high_similarity_questions])
        vectorized_text_with_stopwords = vectorizer.transform([processed_text])
        final_similarities = cosine_similarity(vectorized_text_with_stopwords, Z)
        closest_index = final_similarities.argmax()

        return target_answers[closest_index]
    else:
        return "I don't have information on that. Please ask something else."


In [ ]:
import pandas as pd
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')

def preprocess(text):
    if not isinstance(text, str):
        text = str(text)
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = nltk.word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]
    return ' '.join(stemmed_tokens)

def preprocess_with_stopwords(text):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = nltk.word_tokenize(text.lower())
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]
    return ' '.join(stemmed_tokens)

# Assuming your CSV has a column named 'Answers'
questions_df = pd.read_csv("/content/CHATBOT CAPSTONE DATASET.csv", encoding='unicode_escape')
questions_list = questions_df['Questions'].tolist()
answers_list = questions_df['Answers'].tolist()  # Extract answers

vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize)
X = vectorizer.fit_transform([preprocess(q) for q in questions_list])

def get_response(user_query):
    processed_text = preprocess_with_stopwords(user_query)
    vectorized_text = vectorizer.transform([processed_text])
    similarities = cosine_similarity(vectorized_text, X)
    max_similarity = np.max(similarities)

    if max_similarity > 0.6:
        high_similarity_indices = [i for i, s in enumerate(similarities[0]) if s > 0.6]

        # Ensure index is within bounds of answers_list
        high_similarity_indices = [i for i in high_similarity_indices if i < len(answers_list)]

        high_similarity_questions = [questions_list[i] for i in high_similarity_indices]
        target_answers = [answers_list[i] for i in high_similarity_indices]

        Z = vectorizer.transform([preprocess_with_stopwords(q) for q in high_similarity_questions])
        vectorized_text_with_stopwords = vectorizer.transform([processed_text])
        final_similarities = cosine_similarity(vectorized_text_with_stopwords, Z)

        # Check if final_similarities is empty before calling argmax
        if final_similarities.size > 0:
            closest_index = final_similarities.argmax()
            return target_answers[closest_index]
        else:
            return "I don't have information on that. Please ask something else."
    else:
        return "I don't have information on that. Please ask something else."

print("Hello! I'm a assistant chatbot for IIMS Hotel. How can I help you?")
while True:
    user_query = input("You: ")
    if user_query.lower() in ['exit', 'quit', 'bye']:
        print("Chatbot: Goodbye!")
        break
    response = get_response(user_query)
    print(f"Chatbot: {response}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Hello! I'm a assistant chatbot for IIMS Hotel. How can I help you?
Chatbot: The price of Family room is Rs.3500 per night?
Chatbot: We offer 3 different rooms which are Super comfort, Duplex and family.Which room prices would you want to know?
Chatbot: I don't have information on that. Please ask something else.
Chatbot: Its a budget-friendly room with twin beds and basic amenities.
Chatbot: Hello! Do you have any inquries?
